In [ ]:
pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 51 kB 236 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=2cc434018a626baff1b557d44bed5b3e6e0c8f08c432f2a6cb7ea45bfa5e9569
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("ZNTUF6AFOSTCWEQJUPP66JO7-VUEZTCI3SSJK7BCI")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
from PIL import Image
import matplotlib.pyplot as pyplot
import numpy as np
import itertools

In [ ]:
img = Image.open('./shiva.jpg')

In [ ]:
def resize(img):
  dim = np.ceil(np.sqrt(img.size[0] * img.size[1])).astype(int)
  img = img.resize((dim, dim))
  return img
img = resize(img)


In [ ]:
# img = img.resize((300, 300))
pyplot.imshow(img)
pyplot.show()

In [ ]:

width, height = img.size
window_size = 15
new_img = Image.new('RGB', img.size)
grid = itertools.product(range(0, height-height%window_size, window_size), range(0, width-width%window_size, window_size))
for i, j in grid:
  box = (j, i, j+window_size, i+window_size)
  # print(box)
  # print(j ,i ,j+window_size, i+window_size)
  c_img = img.crop(box)
  # print(np.average(c_img, axis = (0,1)))
  avg_color = np.average(c_img, axis = (0,1))
  # print("avg_color" , avg_color)
  for m in range(box[0], box[2]):
    for n in range(box[1], box[3]):
      new_img.putpixel((m,n), (int(avg_color[0]), int(avg_color[1]), int(avg_color[2])))

pyplot.imshow(new_img)
pyplot.show()

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files

def edge_mask(img, line_size, blur_value):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray_blur = cv2.medianBlur(gray, blur_value)
  edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
  return edges

def color_quantize(img, k):
  # Transform the image
  data = np.float32(img).reshape((-1, 3))

  # Determine criteria
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

  # Implementing K-Means
  ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
  center = np.uint8(center)
  result = center[label.flatten()]
  result = result.reshape(img.shape)
  return result

def cartoonize(img ,blur_value, line_size, colors):
  cv_img = np.array(img)
  cv_img = cv_img[:, :, ::-1].copy()
  # line_size = 5
  # blur_value = 10
  edges = edge_mask(cv_img, line_size, blur_value)
  # colors = 3
  colored = color_quantize(cv_img, colors)
  blurred = cv2.bilateralFilter(colored, d=3, sigmaColor=200,sigmaSpace=200)
  cartoon = cv2.bitwise_and(blurred, blurred, mask=edges)
  cv2_imshow(cartoon)
  cartoon = cv2.cvtColor(cartoon, cv2.COLOR_BGR2RGB)
  return Image.fromarray(cartoon)



# cartoonize(img)



In [ ]:
from PIL import Image
import matplotlib.pyplot as pyplot
import numpy as np
import itertools

def pixelate(img, window_size, blur_value, line_size, colors):
  width, height = img.size
  # window_size = 15
  new_img = Image.new('RGB', img.size)
  img = cartoonize(img, blur_value, line_size, colors)
  grid = itertools.product(range(0, height-height%window_size, window_size), range(0, width-width%window_size, window_size))
  for i, j in grid:
    box = (j, i, j+window_size, i+window_size)
    # print(box)
    # print(j ,i ,j+window_size, i+window_size)
    c_img = img.crop(box)
    # print(np.average(c_img, axis = (0,1)))
    avg_color = np.average(c_img, axis = (0,1))
    # print("avg_color" , avg_color)
    for m in range(box[0], box[2]):
      for n in range(box[1], box[3]):
        new_img.putpixel((m,n), (int(avg_color[0]), int(avg_color[1]), int(avg_color[2])))
  return new_img

In [ ]:
import anvil.media
import io
@anvil.server.callable
def pixelate_img(file, steps, blur_value, line_size, colors):
  with anvil.media.TempFile(file) as file_name:
    img = Image.open(file_name)
    img = pixelate(img, steps, blur_value, line_size, colors)
    byte_arr = io.BytesIO()
    img.save(byte_arr, format='PNG')
    return anvil.BlobMedia('image/png', byte_arr.getvalue(), name='pixel_image.png')

